# Сравнение моделей для предсказания выживаемости пассажиров “Титаника”

## Введение

__Задача__ предсказания выживаемости пассажиров легендарного корабля “Титаник” является классической проблемой бинарной классификации. 

__Цель__ – на основе информации о пассажире (таких как пол, возраст, класс каюты и др.) предсказать, выжил ли он (Survived = 1) или нет (Survived = 0). В данном исследовании проводится сравнительный анализ качества нескольких моделей машинного обучения и глубинного обучения на этом наборе данных. Рассматриваются по одной модели из разных семейств алгоритмов:
* Логистическая регрессия – линейная модель классификации.
* Решающее дерево – модель на основе деревьев решений.
* Случайный лес – ансамблевая модель, усредняющая предсказания множества деревьев.
* XGBoost (градиентный бустинг) – модель градиентного бустинга деревьев решений.

Многослойный перцептрон (MLP) – простой нейросетевой классификатор.
Для каждой модели проводится одинаковый эксперимент: 5-кратная кросс-валидация на обучающем наборе данных с расчетом метрик Accuracy, Precision, Recall, F1-score и ROC-AUC. Далее результаты моделей сравниваются визуально и численно, определяется лучшая модель по совокупности метрик, формулируются выводы и рекомендации.

## Описание данных и подготовка данных

In [1]:
import pandas as pd

In [4]:
# Загрузка данных
df = pd.read_csv('./datasets/feature_titanic_dataset.csv')
print("Размер набора данных:", df.shape)
df.head(5)

Размер набора данных: (891, 15)


,Survived,Pclass,Sex,Age,Fare,FamilySize,IsAlone,Embarked_C,Embarked_Q,Title_Miss,Title_Mrs,Title_Master,Title_Rare,Family_Small,Family_Large
0,0,3,0,22.0,7.2500,2,0,0,0,0,0,0,0,1,0
1,1,1,1,38.0,71.2833,2,0,1,0,0,1,0,0,1,0
2,1,3,1,26.0,7.9250,1,1,0,0,1,0,0,0,0,0
3,1,1,1,35.0,53.1000,2,0,0,0,0,1,0,0,1,0
4,0,3,0,35.0,8.0500,1,1,0,0,0,0,0,0,0,0


Данные не содержат пропущенных значений и уже готовы для моделирования. Тем не менее, необходимо обратить внимание на масштабирование признаков. Логистическая регрессия и нейронная сеть (MLP) чувствительны к масштабу переменных, поэтому для этих моделей мы применим стандартизацию (StandardScaler) к признакам. Для деревьев решений, случайного леса и XGBoost масштабирование не требуется, поскольку эти модели не чувствительны к масштабу входных признаков.

## Методика эксперимента

Для оценки моделей используется метод k-кратной кросс-валидации (k=5). Данные случайным образом делятся на 5 частей (фолдов); в каждом из 5 испытаний 4 фолда используются для обучения модели, а оставшийся фолд – для проверки. Метрики рассчитываются на каждом из 5 контрольных фолдов, после чего усредняются. Такая процедура позволяет получить более надежную оценку обобщающей способности моделей на независимых данных по сравнению с единичным разбиением на обучение/тест. 

__Модели для сравнения__: Мы рассматриваем следующие модели с параметрами по умолчанию (если не указано иначе), каждая настроена для бинарной классификации задачи: логистическая регрессия, решающее дерево, случайный лес, XGBoost и MLP. Для воспроизводимости зададим random_state=42 в моделях, где это применимо. Модели логистической регрессии и MLP будут встроены в Pipeline вместе с StandardScaler для масштабирования признаков. Метрики оценки: Для каждой модели вычисляются следующие показатели качества классификации:
* Accuracy (точность классификации) – доля правильно классифицированных примеров от общего числа.
* Precision (точность позитивного прогноза) – доля правильных предсказаний класса “выжил” среди всех предсказаний “выжил”. Показывает, насколько надежны позитивные прогнозы модели.
* Recall (полнота, чувствительность) – доля реально выживших пассажиров, которых модель правильно идентифицировала как выживших. Характеризует способность модели обнаруживать положительный класс.
* F1-score – гармоническое среднее Precision и Recall, интегральная метрика баланса точности и полноты.
* ROC-AUC – площадь под ROC-кривой, отражающая качество ранжирования: вероятность, что модель присвоит случайному выжившему пассажиру более высокий скор предсказания, чем случайному невыжившему.

Метрики Precision, Recall и F1 рассчитываются относительно положительного класса (выживания). Высокие значения этих метрик указывают на лучшее качество модели по соответствующим критериях.

## Обучение моделей и оценка по метрикам

In [8]:
!pip install xgboost

  Using cached xgboost-3.0.0-py3-none-win_amd64.whl.metadata (2.1 kB)
Using cached xgboost-3.0.0-py3-none-win_amd64.whl (150.0 MB)


In [9]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [10]:
# Формирование признаков X и целевой переменной y
X = df.drop('Survived', axis=1)
y = df['Survived']

In [11]:
# Определение моделей
models = {
    "Logistic Regression": Pipeline([
        ("scaler", StandardScaler()), 
        ("clf", LogisticRegression(max_iter=1000, random_state=42))
    ]),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "Neural Network (MLP)": Pipeline([
        ("scaler", StandardScaler()), 
        ("clf", MLPClassifier(max_iter=500, random_state=42))
    ])
}

In [19]:
# Оценка моделей с помощью 5-fold кросс-валидации
scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
cv_results = {}

# Создаем DataFrame для результатов
results_table = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-score', 'ROC-AUC'])

for name, model in models.items():
    scores = cross_validate(model, X, y, cv=5, scoring=scoring)
    cv_results[name] = {metric: scores[f'test_{metric}'] for metric in scoring}
    
    # Вычисляем среднее и стандартное отклонение для каждой метрики
    accuracy = f"{scores['test_accuracy'].mean():.3f} ± {scores['test_accuracy'].std():.3f}"
    precision = f"{scores['test_precision'].mean():.3f} ± {scores['test_precision'].std():.3f}"
    recall = f"{scores['test_recall'].mean():.3f} ± {scores['test_recall'].std():.3f}"
    f1 = f"{scores['test_f1'].mean():.3f} ± {scores['test_f1'].std():.3f}"
    roc_auc = f"{scores['test_roc_auc'].mean():.3f} ± {scores['test_roc_auc'].std():.3f}"
    
    # Создаем временный DataFrame с результатами одной модели
    temp_df = pd.DataFrame([[name, accuracy, precision, recall, f1, roc_auc]], 
                           columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-score', 'ROC-AUC'])
    
    # Добавляем результаты в общую таблицу
    results_table = pd.concat([results_table, temp_df], ignore_index=True)

# Вывод таблицы
print(results_table.to_markdown(index=False))

c:\Users\bigmi\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:05:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\bigmi\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:05:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\bigmi\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:05:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\bigmi\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:05:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtr

| Model                | Accuracy      | Precision     | Recall        | F1-score      | ROC-AUC       |
|:---------------------|:--------------|:--------------|:--------------|:--------------|:--------------|
| Logistic Regression  | 0.824 ± 0.023 | 0.790 ± 0.034 | 0.739 ± 0.069 | 0.762 ± 0.040 | 0.870 ± 0.025 |
| Decision Tree        | 0.777 ± 0.028 | 0.715 ± 0.042 | 0.699 ± 0.041 | 0.706 ± 0.036 | 0.765 ± 0.038 |
| Random Forest        | 0.806 ± 0.030 | 0.762 ± 0.034 | 0.719 ± 0.075 | 0.738 ± 0.049 | 0.856 ± 0.038 |
| XGBoost              | 0.810 ± 0.024 | 0.767 ± 0.022 | 0.725 ± 0.068 | 0.744 ± 0.043 | 0.861 ± 0.033 |
| Neural Network (MLP) | 0.808 ± 0.025 | 0.798 ± 0.030 | 0.669 ± 0.063 | 0.727 ± 0.045 | 0.855 ± 0.042 |


c:\Users\bigmi\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


## Выводы

По совокупности метрик лучшей моделью на данном датасете оказалась логистическая регрессия – она имеет наивысшие или близкие к наивысшим показатели Accuracy, Recall, F1 и ROC-AUC, одновременно демонстрируя стабильность результатов на разных фолдах. Модель XGBoost следует сразу за ней, показывая сопоставимое качество (особенно по таким метрикам как Precision и ROC-AUC) – разница между этими моделями невелика. Немного уступает им случайный лес, однако он все же значительно лучше, чем одно решающее дерево, которое в данном эксперименте оказалось худшим. MLP-модель не превосходила лучшие алгоритмы и показала смещенность в сторону Precision ценой потери Recall, что не оптимально, если цель – находить всех выживших.